# **Introduction to EDA/Modelling for Multi-Class Prediction of Obesity Risk**

![easy_peasy_EDA](https://th.bing.com/th/id/OIG4.Z3X2KIgpwXplIxzX6Z90?pid=ImgGn)

The notebook has the **Multiclass Classification with a Obesity Risk competition, Playground Season-4 Episode-2**, as object.

The notebook is structured as a series of cells and comments, from data exploration to building and optimizing a first starter model using XGBoost. By following it you can understabnd both the data problem and grasp the actions to be taken with similar tabular data problems.

The approach is to explore data using the essential commands for tabular and chart representations. Often, on Kaggle Notebooks for competitions, EDA is weighted with tons of charts and plots that you cannot understand what they serve for, based on the principle "The More The Merrier". The idea is instead to just go to the point and show the relevant plots and keyt understanding in a dataset that could help you to devise the better strategies and feature engineering.

The modelling part is also striving to just do the essential, hence you will find a single model devised to show how far you can go with the state of the art in tabular data modelling, i.e. XGBoost.

The notebook is arrange into six parts, with sections:

1. Data loading and first exploration
2. Target analysis
3. EDA and data preparation
4. Modelling
5. Explainability
6. Preparation of the submission

Most of the used approaches / code snippets can be found on the books:

1. The Kaggle Book by Konrad Banachewicz and Luca Massaron
2. The Kaggle Workbook by Konrad Banachewicz and Luca Massaron
3. Developing Kaggle Notebooks by Gabriel Preda

for a wider discussion of the topics of EDA and modelling just refer to these books

## 1.0 Data loading and first exploration

First, the basic Python packages (more will be loaded later, when needed)

In [ ]:
import numpy as np
import pandas as pd

Load the data and set the id as index

In [ ]:
train = pd.read_csv("/kaggle/input/playground-series-s4e2/train.csv").set_index("id")
test = pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv").set_index("id")

In [ ]:
train.columns

In [ ]:
train.head(5).T

In [ ]:
test.head(5).T

The dataset seems similar to "Obesity based on eating habits & physical cond" (https://www.kaggle.com/datasets/ankurbajaj9/obesity-levels) which has been derived from:

Palechor, Fabio Mendoza, and Alexis de la Hoz Manotas. "Dataset for estimation of obesity levels based on eating habits and physical condition in individuals from Colombia, Peru and Mexico." Data in brief 25 (2019): 104344.
(https://www.sciencedirect.com/science/article/pii/S2352340919306985)

The research paper introduces a dataset designed for estimating obesity levels in individuals across Mexico, Peru, and Colombia. The estimation relies on factors such as eating habits and physical condition. The dataset encompasses 17 attributes and 2111 records, each labeled with the class variable NObesity (Obesity Level). This variable facilitates the classification of data into categories including Insufficient Weight, Normal Weight, Overweight Level I, Overweight Level II, Obesity Type I, Obesity Type II, and Obesity Type III.

Of the total data, 77% was synthetically generated using the Weka tool and the SMOTE filter, while the remaining 23% was directly collected from users through a web platform. The dataset serves as a valuable resource for developing intelligent computational tools capable of identifying an individual's obesity level. Additionally, it can be utilized for constructing recommender systems aimed at monitoring and addressing obesity levels. 

In particular we can deduct that:
* 'Gender', 'Age', 'Height', 'Weight' are physical attributes
* 'family_history_with_overweight' points out to familiarity with obesity
* FAVC = Frequent consumption of high caloric food
* FCVC = Frequency of consumption of vegetables
* NCP = Number of main meals
* CAEC = Consumption of food between meals
* SMOKE = tobacco usage
* CH20 = Consumption of water daily
* SCC = Calories consumption monitoring
* FAF = Physical activity frequency
* TUE = Time using technology devices
* CALC = Consumption of alcohol
* MTRANS = Transportation used

The target is estracted from the train

In [ ]:
target = train["NObeyesdad"]
train = train.drop("NObeyesdad", axis="columns")

## 2.0 Target analysis

By examining the distribution of the target we can figure that: 

1. it is a multiclass problem, with 7 class
2. the classes are distributed differently but there are not extreme differences in their proportions (expected probability)
3. However, the most frequent class (Obesity_Type_III) has almost the double of probability of the least frequent one (Overweight_Level_I), hence, when defining the cross-validation strategy when should consider a target stratified one thus such different prior probabilities will be reflected also in the test set as much exactly as possible.

In [ ]:
target.value_counts()

In [ ]:
target.value_counts() / len(target)

Here we represent the situation with a plot, ordering the classes based on overweight

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

color_list = ["#A5D7E8", "#576CBC", "#19376D", "#0B2447"]
cmap_custom = ListedColormap(color_list)

sorted_labels = ['Insufficient_Weight', 'Normal_Weight', 
 'Overweight_Level_I', 'Overweight_Level_II', 
 'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III']

plt.figure(figsize=(8, 4))
ax = sns.countplot(x=target, order=sorted_labels, palette=color_list)

plt.title('Distribution of Obesity Risk Levels')
plt.xlabel('Obesity Risk Levels')
plt.ylabel('Count')

ax.set_xticklabels(ax.get_xticklabels(), rotation=10, ha='right', fontsize=8)
plt.tight_layout() 
plt.show()

for modelling purposes, we convert the target, expressed by string labels, into numeric labels, using a custom implementation of Scikit-learn LabelEncoder that orders the encoded label based on the order we proposed before.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CustomLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, sorted_labels):
        self.classes_ = sorted_labels
        self.label_to_numeric_mapping = None

    def fit(self, y):
        self.label_to_numeric_mapping = {label: numeric for numeric, label in enumerate(self.classes_)}
        return self

    def transform(self, y):
        if self.label_to_numeric_mapping is None:
            raise ValueError("fit method must be called before transform")
        return y.map(self.label_to_numeric_mapping)

    def inverse_transform(self, y):
        if self.label_to_numeric_mapping is None:
            raise ValueError("fit method must be called before inverse_transform")
        return pd.Series(y).map({numeric: label for label, numeric in self.label_to_numeric_mapping.items()})

In [ ]:
target_encoder = CustomLabelEncoder(sorted_labels)
target_encoder.fit(target)
target_numeric = target_encoder.transform(target)

In [ ]:
target_numeric

Using our enconder, we can rever back from numeric to string anytime:

In [ ]:
target_encoder.inverse_transform(target_numeric)

## 3.0 EDA and data preparation

Let's proceed to indeep explore and prepare the datasets (both the train and the test in parallel) for being suitable for modelling with both GLMs and Tree-based models

### 3.1 Missing values

Both train and test have no missing data

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

### 3.2 Data Types and feature classification

Let's enumerate the numeric (continuous and ordinal) features fromt the categorical ones

In [ ]:
train.dtypes

In [ ]:
categorical_features = train.columns[train.dtypes=="object"].tolist()
numeric_features = train.columns[train.dtypes!="object"].tolist()

All the categorical features have very few unique levels, making one-hot-encoding the best choice for this situation

In [ ]:
train[categorical_features].nunique()

All the numeric features, have enough distinct values to confirm their are continuous ones

In [ ]:
train[numeric_features].nunique()

### 3.3 Plotting count and distribution pairs

In [ ]:
def plot_count_pairs(train, test, feature, hue="set", order=None, palette=None):
    data_df = train.copy()
    data_df['set'] = 'train'
    data_df = pd.concat([data_df, test.copy()]).fillna('test')
    f, ax = plt.subplots(1, 1, figsize=(8, 4))
    sns.countplot(x=feature, data=data_df, hue=hue, palette=color_list, order=order)
    plt.grid(color="black", linestyle="-.", linewidth=0.5, axis="y", which="major")
    ax.set_title(f"Paired train/test frequencies of {feature}")
    plt.show()
    
for feature in categorical_features:
    if feature in ["CAEC", "CALC"]:
        order = ["no", "Always", "Sometimes", "Frequently"]
    else:
        order = sorted(train[feature].unique())
    plot_count_pairs(train, test, feature=feature, order=order, palette=color_list)

In [ ]:
import warnings

def plot_distribution_pairs(train, test, feature, hue="set", palette=None):
    data_df = train.copy()
    data_df['set'] = 'train'
    data_df = pd.concat([data_df, test.copy()]).fillna('test')
    data_df.replace([np.inf, -np.inf], np.nan, inplace=True)

    f, ax = plt.subplots(1, 1, figsize=(8, 4))
    for i, s in enumerate(data_df[hue].unique()):
        selection = data_df.loc[data_df[hue]==s, feature]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=FutureWarning)
            g = sns.histplot(selection, color=palette[i], ax=ax, label=s)
    ax.set_title(f"Paired train/test distributions of {feature}")
    g.legend()
    plt.show()

def plot_distribution_pairs_boxplot(train, test, feature, hue="set", palette=None):
    data_df = train.copy()
    data_df['set'] = 'train'
    data_df = pd.concat([data_df, test.copy()]).fillna('test')
    data_df.replace([np.inf, -np.inf], np.nan, inplace=True)

    f, ax = plt.subplots(1, 1, figsize=(8, 4))
    for i, s in enumerate(data_df[hue].unique()):
        selection = data_df.loc[data_df[hue]==s, feature]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=FutureWarning)
            g = sns.boxplot(x=hue, y=feature, data=data_df, palette=palette, ax=ax)
    ax.set_title(f"Paired train/test boxplots of {feature}")
    g.legend()
    plt.show()
    
for feature in numeric_features:
    plot_distribution_pairs(train, test, feature, palette=color_list)
    plot_distribution_pairs_boxplot(train, test, feature, palette=color_list)

### 3.4 Categorical encoding

We then proceed to process the categorical features by encoding them using the OneHotEncoder by Scikit-learn

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
encoder.fit(pd.concat([train[categorical_features], test[categorical_features]], axis=0))

train_encoded = encoder.transform(train[categorical_features])
train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(categorical_features))

In [ ]:
test_encoded = encoder.fit_transform(test[categorical_features])
test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(categorical_features))

In [ ]:
train_sum = train_encoded_df.sum(axis=0).astype(int)
test_sum = test_encoded_df.sum(axis=0).astype(int)

sum_df = pd.concat([train_sum, test_sum], axis=1, keys=['Train', 'Test'])
sum_df

### 3.5 Adjusting categorical encodings

CALC_Always, being zero in training won't be used and there could be problems in the test if CALC is a relevant feature. We can then perform the combination of columns CALC_Always and CALC_Frequently in both train_encoded_df and test_encoded_df into a new column called 'CALC_Always|Frequently'. After creating the combined column, the original columns CALC_Always and CALC_Frequently are dropped.

In [ ]:
combine_columns = ['CALC_Always', 'CALC_Frequently']

train_encoded_df['CALC_Always|Frequently'] = train_encoded_df[combine_columns].sum(axis=1)
test_encoded_df['CALC_Always|Frequently'] = test_encoded_df[combine_columns].sum(axis=1)

train_encoded_df = train_encoded_df.drop(columns=combine_columns).set_index(train.index)
test_encoded_df = test_encoded_df.drop(columns=combine_columns).set_index(test.index)

Before proceeding, let's also consider that CALC and CAEC are indeed ordinal features. We can replace them with an ordinal feature, too, to catch any linear effect due to these two features. 

In [ ]:
levels = {"Always": 3, "Frequently": 2, "Sometimes": 1, "no": 0}
train["CALC_ord"] = train["CALC"].map(levels)
test["CALC_ord"] = test["CALC"].map(levels)
train["CAEC_ord"] = train["CAEC"].map(levels)
test["CAEC_ord"] = test["CAEC"].map(levels)

Let's now integrate the categorical encodings in our original train and test datasets, replacing the previous categorical 
features

In [ ]:
train = pd.concat([train.drop(categorical_features, axis=1), train_encoded_df], axis=1)
test = pd.concat([test.drop(categorical_features, axis=1), test_encoded_df], axis=1)

As a furthermore check before proceeding modelling, let's test adversarial validation between the train and the test datasets in order to be sure that they are from the same distribution and there is no particular shift between them. This will ensure us that we should expect the same target distribution too.

### 3.6 Adversarial validation

We take the code for adversarial validation from the Kaggle Book: https://github.com/PacktPublishing/The-Kaggle-Book/blob/main/chapter_06/adversarial-validation-example.ipynb

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score

X =  pd.concat([train, test], axis=0)
y = [0] * len(train) + [1] * len(test)

model = RandomForestClassifier(random_state=0)
cv_preds = cross_val_predict(model, X, y, cv=5, n_jobs=-1, method='predict_proba')

score = roc_auc_score(y_true=y, y_score=cv_preds[:,1])
print(f"roc-auc score: {score:0.3f}")

Since the roc-auc score is about 0.5, we have a confirmation that train and test are from the same distribution.

### 3.7 Feature engineering

At this point we try to create some smart feature engineering. As a first step we calculate the Body Mass Index using the 'Height' and 'Weight' features. BMI is often used as an indicator of obesity and it better represents the relationship between weight and height. 

In [ ]:
train['BMI'] = train['Weight'] / (train['Height'] ** 2)
test['BMI'] = test['Weight'] / (test['Height'] ** 2)

In [ ]:
plot_distribution_pairs(train, test, feature="BMI", palette=color_list)

We then combine 'FAF' (Physical activity frequency) with 'TUE' (Time using technology devices) to create a feature representing the balance between physical activity and sedentary behavior.

In [ ]:
train['Physical_Activity_Level'] = train['FAF'] - train['TUE']
test['Physical_Activity_Level'] = test['FAF'] - test['TUE']

In [ ]:
plot_distribution_pairs(train, test, feature="Physical_Activity_Level", palette=color_list)

Combine 'FCVC' (Frequency of consumption of vegetables) and 'NCP' (Number of main meals) to create a feature reflecting overall meal habits.

In [ ]:
train['Meal_Habits'] = train['FCVC'] * train['NCP']
test['Meal_Habits'] = test['FCVC'] * test['NCP']

In [ ]:
plot_distribution_pairs(train, test, feature="Meal_Habits", palette=color_list)

We also combine features like 'FCVC' (Frequency of consumption of vegetables) and 'FAVC' (Frequent consumption of high caloric food) to create a new feature representing overall nutrition habits.

In [ ]:
(train['FAVC_yes'] - 2).unique

In [ ]:
train['Healthy_Nutrition_Habits'] = train['FCVC'] / ( 2 * train['FAVC_no'] - 1)
test['Healthy_Nutrition_Habits'] = test['FCVC'] / ( 2 * test['FAVC_no'] - 1)

In [ ]:
plot_distribution_pairs(train, test, feature="Healthy_Nutrition_Habits", palette=color_list)

We finally create a score based on the frequency of technology usage ('TUE') weighted by the age: essentially, it quantifies the average amount of time a person spends using technology per unit of their age

In [ ]:
train['Tech_Usage_Score'] = train['TUE'] / train['Age']
test['Tech_Usage_Score'] = test['TUE'] / test['Age']

In [ ]:
plot_distribution_pairs(train, test, feature="Tech_Usage_Score", palette=color_list)

Since this completes our data preparation, we store the resulting transformed train and test datasets, hence we can reuse them in other Kaggle Notebooks.

In [ ]:
train['Tech_Usage_Score'] = train['TUE'] / train['Age']
test['Tech_Usage_Score'] = test['TUE'] / test['Age']

In [ ]:
train.to_csv("train.csv")
test.to_csv("test.csv")
target.to_csv("target.csv")

## 4.0 Modelling

We can now proceed to modelling. First by defining our cv strategy, which is, as mentioned before, a stratified cross-validation based on the distribution of target labels.

In [ ]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(5, shuffle=True, random_state=0)
cv_splits = cv.split(train, y=target_numeric)

In [ ]:
list(cv_splits)

### 4.1 First temptative models (hand tuning)

XGBoost, by default, constructs an individual model for each target, a behavior reminiscent of meta-estimators in scikit-learn. This approach offers the advantage of efficient data reuse and leverages integrated features such as SHAP (SHapley Additive exPlanations). In the context of multi-label classification, XGBoost employs the binary relevance strategy. This strategy involves building separate binary classifiers for each class, treating the classification problem as a set of independent binary tasks. This allows the model to handle multi-label scenarios where each instance may belong to multiple classes.

In [ ]:
%%time

from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score
from xgboost import XGBClassifier

xgb = XGBClassifier(booster='gbtree',
                    device = "cuda",
                    tree_method="hist",
                    objective='multi:softmax',  # 'multi:softmax' for multiclass classification
                    num_class=len(set(target_numeric)),  # Number of classes
                    n_estimators=300,
                    max_depth=6,
                    verbosity=0)

cv_splits = cv.split(train, y=target_numeric)
scorer = make_scorer(accuracy_score)

cv_results = cross_validate(xgb, train, target_numeric, cv=cv_splits, scoring=scorer, return_train_score=False)

accuracy_mean = cv_results['test_score'].mean()
print(f'Cross-validated Accuracy: {accuracy_mean:.3f}')

One helpful step, after creating the first temptative model, is to do some error analysis and try to understand what is not working in the approach. To do so, we need to resort to cross_val_predict which will create a cross validated prediction useful to check the fit of the model.? 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict

cv_splits = cv.split(train, y=target_numeric)
cv_cls_preds = cross_val_predict(xgb, train, target_numeric, cv=cv_splits)

In [ ]:
print(classification_report(y_true=target_numeric, y_pred=cv_cls_preds, target_names=target_encoder.classes_))

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_true=target_numeric, y_pred=cv_cls_preds)

plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2) 
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap=cmap_custom, 
            xticklabels=sorted_labels, yticklabels=sorted_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

From the overview of the error analysis, a step that always to be taken when working with tabular data, it seems that the decision boundaries between certain categories, in particular Overweight_Level_I and Overweight_Level_II, are not so well defined. In such cases it is helpful to use blended submissions between different but equally predictive models, to have better shaped class boundaries.

A solution, in our case, would be to blend our XGBoost with another gradient boosting solution, such as a LightGBM.

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(boosting_type="gbdt",
                     objective="multiclass",
                     metric="multi_logloss",
                     num_class=7,
                     learning_rate=0.025,
                     n_estimators=500,
                     lambda_l1=0.06,
                     lambda_l2=0.3,
                     max_depth=9,
                     colsample_bytree=0.40,
                     subsample=0.85,
                     min_child_samples=15,
                     verbosity=-1)

cv_splits = cv.split(train, y=target_numeric)

cv_results = cross_validate(lgb, train, target_numeric, cv=cv_splits, scoring=scorer, return_train_score=False)

accuracy_mean = cv_results['test_score'].mean()
print(f'Cross-validated Accuracy: {accuracy_mean:.3f}')

In [ ]:
cv_splits = cv.split(train, y=target_numeric)
cv_cls_lgb_preds = cross_val_predict(lgb, train, target_numeric, cv=cv_splits)

print(classification_report(y_true=target_numeric, y_pred=cv_cls_lgb_preds, target_names=target_encoder.classes_))

conf_matrix = confusion_matrix(y_true=target_numeric, y_pred=cv_cls_lgb_preds)

plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2) 
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap=cmap_custom, 
            xticklabels=sorted_labels, yticklabels=sorted_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

improvement = np.sum((cv_cls_lgb_preds==target_numeric)&(cv_cls_preds!=target_numeric)) / len(target_numeric)
print(f"maximum possible improvement: {improvement:0.3f}")

training a LightGBM and doing some error analysis, does perform slightly better than XGBoost on the problematic classes. An estimation of possible maximum improvement considering only the correct estimates (not the erroneous ones) that a LightGBM could bring in a blend seems promising.

Before proceeding, however, let's test how the new multi output tree, which is especially designed for multiclass problem, could work.

As of XGBoost version 2.0, in fact, a new multi_strategy option has become available. the training parameter called multi_strategy allows you to specify how the model should handle multiple targets when constructing trees.

By default, when multi_strategy is set to "one_output_per_tree" (which is the default), XGBoost builds one model for each target. Each model is trained independently for its corresponding target.

Alternatively, when multi_strategy is set to "multi_output_tree," XGBoost constructs multi-output trees. In this mode, each leaf node of the tree is associated with all the targets, and the model learns to make joint predictions for multiple targets simultaneously. This allows to build a single, compact model.

Since the multi_output_tree option is a work in progress, at the moment it only supports cpu as a device.

In [ ]:
%%time

xgb = XGBClassifier(booster='gbtree',
                    device = "cpu",
                    tree_method="hist",
                    multi_strategy="multi_output_tree", # for multi-output trees
                    objective='multi:softmax',  # 'multi:softmax' for multiclass classification
                    num_class=len(set(target_numeric)),  # Number of classes
                    n_estimators=300,
                    max_depth=6,
                    verbosity=0)

cv_splits = cv.split(train, y=target_numeric)

cv_results = cross_validate(xgb, train, target_numeric, cv=cv_splits, scoring=scorer, return_train_score=False)

accuracy_mean = cv_results['test_score'].mean()
print(f'Cross-validated Accuracy: {accuracy_mean:.3f}')

The solution is not bad, let's go for the faster binary relevance strategy, hence we can leverage the GPU, but let's consider the multi_output_strategy for a blended solution.

In [ ]:
cv_splits = cv.split(train, y=target_numeric)
cv_cls_multi_preds = cross_val_predict(xgb, train, target_numeric, cv=cv_splits)

print(classification_report(y_true=target_numeric, y_pred=cv_cls_multi_preds, target_names=target_encoder.classes_))

conf_matrix = confusion_matrix(y_true=target_numeric, y_pred=cv_cls_multi_preds)

plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2) 
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap=cmap_custom, 
            xticklabels=sorted_labels, yticklabels=sorted_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
improvement = np.sum((cv_cls_multi_preds==target_numeric)&(cv_cls_preds!=target_numeric)) / len(target_numeric)
print(f"maximum possible improvement: {improvement:0.3f}")

### 4.2 Approching the problem as a regression

Since the target is ordinal, we can test also operating using a regression instead of a classification

In [ ]:
from xgboost import XGBRegressor

cv_splits = cv.split(train, y=target_numeric)
cv_scores = list()
cv_train_preds = np.zeros(len(train))
cv_test_preds = np.zeros(len(test))

for train_idx, val_idx in cv_splits:
    xgb = XGBRegressor(booster='gbtree',
                       device = "cpu",
                       tree_method="hist",
                       n_estimators=200,
                       max_depth=4,
                       verbosity=0)
    xgb.fit(train.iloc[train_idx], target_numeric.iloc[train_idx])
    preds = xgb.predict(train.iloc[val_idx])
    cv_train_preds[val_idx] = preds
    cv_test_preds += xgb.predict(test) / cv.n_splits
    cls_preds = np.clip(np.round(preds).astype(int), a_min=0, a_max=6)
    
    cv_scores.append(accuracy_score(y_true=target_numeric.iloc[val_idx], y_pred=cls_preds))

accuracy_mean = np.mean(cv_scores)
print(f'Cross-validated Accuracy: {accuracy_mean:.3f}')

In [ ]:
print(classification_report(y_true=target_numeric, 
                            y_pred=np.clip(np.round(cv_train_preds).astype(int), a_min=0, a_max=6), 
                            target_names=target_encoder.classes_))

The problems with this predictor are again the first two overweight categories, it tend to appear a weaker solution though, than using multiclass classification. 

In [ ]:
conf_matrix = confusion_matrix(y_true=target_numeric, 
                               y_pred=np.clip(np.round(cv_train_preds).astype(int), a_min=0, a_max=6))

plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2) 
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap=cmap_custom, 
            xticklabels=sorted_labels, yticklabels=sorted_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

The results from the regression are disappointing, but the regression's results could be improved and possibly be used as a feature for the classification.

### 4.3 Model optimization by Optuna and Bayesian search

As a next step we optimize XGBoost most important hyper-parameters using a bayesian search by Optuna. The TPESampler is set to first try random optimization for the first 30 trials and then perform a TPE Bayesian Search. We also use the multivariate approach (added in v2.2.0 as an experimental feature), which is reported to outperform the independent TPE.

In [ ]:
import optuna
from optuna.samplers import TPESampler

def objective(trial):
    
    params = {
        'grow_policy': trial.suggest_categorical('grow_policy', ["depthwise", "lossguide"]),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log=True),
        'gamma' : trial.suggest_float('gamma', 1e-9, 0.5),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'max_depth': trial.suggest_int('max_depth', 0, 16),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 100.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 100.0, log=True),
        
    }
    
    params['booster'] = 'gbtree'
    params['objective'] = 'multi:softmax'
    params["device"] = "cuda"
    params["verbosity"] = 0
    params['tree_method'] = "hist"
    
    xgb = XGBClassifier(**params)
    cv_splits = cv.split(train, y=target_numeric)

    cv_scores = cross_validate(estimator=xgb, 
                               X=train, 
                               y=target_numeric,
                               scoring=scorer,
                               cv=cv_splits)

    cv_evaluation = np.mean(np.abs(cv_scores['test_score']))
    return cv_evaluation

sqlite_db = "sqlite:///sqlite.db"
study_name = "multi_class_prediction_of_obesity_risk"
study = optuna.create_study(storage=sqlite_db, study_name=study_name, 
                            sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=0),
                            direction="maximize", load_if_exists=True)

study.optimize(objective, n_trials=150)
print(f"best optmized accuracy: {study.best_value:0.5f}")
print(f"best hyperparameters: {study.best_params}")

best_cls_params = study.best_params
best_cls_params['objective'] = 'multi:softmax'
best_cls_params['tree_method'] = "hist"
best_cls_params["device"] = "cuda"
best_cls_params["verbosity"] = 0

### 4.4 Re-train on all data

Once we obtained the best parameters, we re-train on all our available data

In [ ]:
xgb = XGBClassifier(**best_cls_params)
xgb.fit(train, target_numeric)

The we proceed to inference, convert outputted numeric labels into strings and then save everything into a submission

## 5.0 Explainability

To understand how the model actually works, the XGBoost model itself, represented by the booster, is used to calculate SHAP (SHapley Additive exPlanations) values for each feature in the training data. The feature names are extracted from the 'train' DataFrame, and a DMatrix 'Xd' is created to represent the training data in a format suitable for XGBoost. The SHAP values are obtained by predicting contributions to the predictions using the 'predict' method with 'pred_contribs=True'. 

In [ ]:
from xgboost import DMatrix

booster = xgb.get_booster()

feature_names = train.columns
Xd = DMatrix(train)

shap_values = booster.predict(Xd, pred_contribs=True)
preds = booster.predict(Xd)

then we create a summary plot visualizing the impact of different features on the model predictions. The shap.summary_plot function is employed to generate a violin plot, where each violin represents the distribution of SHAP values for a specific feature. 

In [ ]:
shap_values[:,0,:-1].shape

In [ ]:
target_encoder.classes_

In [ ]:
train.shape

In [ ]:
import shap
import matplotlib.pyplot as plt

for k, target_label in enumerate(target_encoder.classes_):
    print(f"VIOLIN PLOT REPRESENTING <{target_label}>")
    shap.summary_plot(shap_values[:,k, :-1], train.values,
                      plot_type="violin",
                      feature_names=feature_names, show=False)
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)
    plt.show()
    print()

## 6.0 Preparation of the submission

In [ ]:
preds = xgb.predict(test)
labelled_preds = target_encoder.inverse_transform(preds)

In [ ]:
submission = pd.read_csv("/kaggle/input/playground-series-s4e2/sample_submission.csv")
submission["NObeyesdad"] = labelled_preds.values
submission.to_csv("xgb_submission.csv", index=False)

In [ ]:
submission.head(5)

## 7.0 Submission strategies

Now that our single model submission is ready, let's prepare an ensembled one with multiout XGBoost and LightGBM and check if it improves our results as we figured after error analysis.

In [ ]:
from copy import copy

multi_cls_params = copy(best_cls_params)
multi_cls_params["device"] = "cpu"
multi_cls_params["tree_method"] = "hist"
multi_cls_params["multi_strategy"] = "multi_output_tree"

In [ ]:
multi = XGBClassifier(**multi_cls_params)
multi.fit(train, target_numeric)

lgb = LGBMClassifier(boosting_type="gbdt",
                     objective="multiclass",
                     metric="multi_logloss",
                     num_class=7,
                     learning_rate=0.025,
                     n_estimators=500,
                     lambda_l1=0.06,
                     lambda_l2=0.3,
                     max_depth=9,
                     colsample_bytree=0.40,
                     subsample=0.85,
                     min_child_samples=15,
                     verbosity=-1)

lgb.fit(train, target_numeric)

xgb_preds = xgb.predict_proba(test)
lgb_preds = lgb.predict_proba(test)
multixgb_preds = multi.predict_proba(test)

blended_preds = np.argmax((xgb_preds + lgb_preds + multixgb_preds), axis=1).ravel()
labelled_blended_preds = target_encoder.inverse_transform(blended_preds)

submission = pd.read_csv("/kaggle/input/playground-series-s4e2/sample_submission.csv")
submission["NObeyesdad"] = labelled_blended_preds.values
submission.to_csv("blended_submission.csv", index=False)

In [ ]:
submission.head(5)

Another strategy is to bag ((Bootstrap Aggregating) multiple times your model and then blend the results. This should result in more stable results in the private leaderboard

In [ ]:
from tqdm import tqdm

boots = 300

for k in tqdm(range(boots)):
    bootstrap_idx = np.random.choice(train.index, size=len(train), replace=True)

    xgb = XGBClassifier(**best_cls_params)
    xgb.fit(train.loc[bootstrap_idx], target_numeric.loc[bootstrap_idx])
    if k==0:
        preds = xgb.predict_proba(test)
    else:
        preds += xgb.predict_proba(test)
        
blended_preds = np.argmax(preds, axis=1).ravel()
labelled_blended_preds = target_encoder.inverse_transform(blended_preds)

submission = pd.read_csv("/kaggle/input/playground-series-s4e2/sample_submission.csv")
submission["NObeyesdad"] = labelled_blended_preds.values
submission.to_csv("bagged_submission.csv", index=False)

In [ ]:
submission.head(5)

Here we complete our overview of this multiclass problem. The next step in the competition is to feed the data we prepared to a auto-ml engine or build a blending or stacking solution that could bring you up the leaderboard. Good luck!